In [108]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [109]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bluff,NZ,2020-07-31 10:20:00,-46.60,168.33,50.13,88,82,4.21
1,1,Meihekou,CN,2020-07-31 10:20:00,42.53,125.68,79.61,70,31,10.04
2,2,Alice Springs,AU,2020-07-31 10:15:10,-23.70,133.88,62.60,14,0,6.93
3,3,Port Hardy,CA,2020-07-31 10:20:00,50.70,-127.42,58.26,85,12,1.36
4,4,Lasa,IT,2020-07-31 10:20:01,46.62,10.70,78.01,68,40,8.05


In [110]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [111]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [112]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [113]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [114]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [115]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [116]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.tail()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
520,520,Katobu,ID,2020-07-31 10:21:15,-4.94,122.53,77.22,73,62,6.69
523,523,Poum,NC,2020-07-31 10:21:15,-20.23,164.02,71.22,88,88,24.87
526,526,Cervo,ES,2020-07-31 10:21:16,43.67,-7.41,75.99,58,14,1.01
534,534,Itaituba,BR,2020-07-31 10:21:17,-4.28,-55.98,73.53,62,73,0.92
543,543,Calabozo,VE,2020-07-31 10:21:18,8.92,-67.43,71.64,97,57,1.88


In [117]:
preferred_cities_df.notnull().sum()

City_ID       158
City          158
Country       157
Date          158
Lat           158
Lng           158
Max Temp      158
Humidity      158
Cloudiness    158
Wind Speed    158
dtype: int64

In [118]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       1
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [119]:
preferred_cities_df.count()

City_ID       158
City          158
Country       157
Date          158
Lat           158
Lng           158
Max Temp      158
Humidity      158
Cloudiness    158
Wind Speed    158
dtype: int64

In [120]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Meihekou,CN,79.61,42.53,125.68,
4,Lasa,IT,78.01,46.62,10.70,
15,Rikitea,PF,72.95,-23.12,-134.97,
17,Saint-Philippe,RE,77.00,-21.36,55.77,
20,Hilo,US,73.40,19.73,-155.09,
21,Alyangula,AU,73.40,-13.85,136.42,
31,Sampit,ID,79.54,-2.53,112.95,
41,Maxixe,MZ,77.00,-23.86,35.35,
42,Warri,NG,77.49,5.52,5.75,
45,Golcuv Jenikov,CZ,75.20,49.82,15.48,


In [121]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [122]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()


In [123]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(50)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Meihekou,CN,79.61,42.53,125.68,如家宾馆
4,Lasa,IT,78.01,46.62,10.70,Pension Feldgärtenhof
15,Rikitea,PF,72.95,-23.12,-134.97,Pension Maro'i
17,Saint-Philippe,RE,77.00,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
20,Hilo,US,73.40,19.73,-155.09,Hilo Hawaiian Hotel
21,Alyangula,AU,73.40,-13.85,136.42,Groote Eylandt Lodge
31,Sampit,ID,79.54,-2.53,112.95,Aquarius Boutique Hotel Sampit
41,Maxixe,MZ,77.00,-23.86,35.35,Oceano
42,Warri,NG,77.49,5.52,5.75,BON Hotel Delta
45,Golcuv Jenikov,CZ,75.20,49.82,15.48,Chateau Hostačov


In [124]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [125]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [126]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [127]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [128]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))